# 제 2유형 연습하기 : iris 종 분류 (이진분류)

## ✅ 데이터 분석 순서
1. 라이브러리 및 데이터 탐색
2. 데이터 탐색 (EDA)
3. 데이터 전처리 및 분리
4. 모델링 및 성능 평가
5. 예측값 제출

### ✅ 1. 라이브러리 및 데이터 탐색

In [54]:
import pandas as pd
import numpy as np

In [55]:
############### 실기환경 복사 영역 ############### 
import pandas as pd
import numpy as np
# 실기 시험 데이터셋으로 셋팅하기 (수정금지)
from sklearn.datasets import load_iris
# Iris 데이터셋을 로드
iris = load_iris()
x = pd.DataFrame(iris.data, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width']) 
y = iris.target # 'setosa'=0, 'versicolor'=1, 'virginica'=2
y = np.where(y>0, 1, 0) # setosa 종은 0, 나머지 종은 1로 변경


# 실기 시험 데이터셋으로 셋팅하기 (수정금지)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2023)

x_test = pd.DataFrame(x_test) 
x_train = pd.DataFrame(x_train) 
y_train = pd.DataFrame(y_train) 
y_train.columns = ['species']

# 결측치 삽입
x_test['sepal_length'].iloc[0] = None 
x_train['sepal_length'].iloc[0] = None

# 이상치 삽입
x_train['sepal_width'].iloc[0] = 150 

############### 실기환경 복사 영역 ###############
### 참고사항 ###
# y_test 는 실기 문제상에 주어지지 않음
# ★Tip : X를 대문자로 쓰지말고 소문자 x로 쓰세요. 시험에서 실수하기 쉽습니다.(문제풀기 전에 소문자로 변경!) # (참고 : 보통 X는 2차원 배열(행렬)이기 때문에 대문자로 쓰고, y는 1차원 배열(벡터)이기 때문에 소문자로 씀)
# (~23년 10월말) 실기시험 데이터 형식 (실제 시험장에서는 다를 수 있으니 반드시 체크) # X_test = pd.read_csv("data/X_test.csv")
# X_train = pd.read_csv("data/X_train.csv")
# y_train = pd.read_csv("data/y_train.csv")
# ★(23년 10월말~) 기준으로 체험환경에서 제공되는 데이터셋이 조금 변경되었습니다. # train = pd.read_csv("data/customer_train.csv")
# test = pd.read_csv("data/customer_test.csv")
# x_train과 y_train, x_test를 별도로 할당해주셔야 합니다.

붓꽃의 종(Species)를 분류해보자.                
- 데이터의 결측치, 이상치에 대해 처리하고               
- 분류 모델을 사용하여 정확도, F1 score, AUC 값을 산출하시오.           
- 제출은 result 변수에 담아 양식에 맞게 제출하시오

In [56]:
# 데이터 설명
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

### ✅ 2. 데이터 탐색 (EDA)

In [57]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(120, 4)
(30, 4)
(120, 1)


In [58]:
# 초기 데이터 확인
print(x_train.head())
print(x_test.head())
print(y_train.head())

     sepal_length  sepal_width  petal_length  petal_width
2             NaN        150.0           1.3          0.2
49            5.0          3.3           1.4          0.2
66            5.6          3.0           4.5          1.5
118           7.7          2.6           6.9          2.3
138           6.0          3.0           4.8          1.8
     sepal_length  sepal_width  petal_length  petal_width
93            NaN          2.3           3.3          1.0
69            5.6          2.5           3.9          1.1
137           6.4          3.1           5.5          1.8
43            5.0          3.5           1.6          0.6
60            5.0          2.0           3.5          1.0
   species
0        0
1        0
2        1
3        1
4        1


In [59]:
# 변수명과 데이터 타입이 매칭이 맞는지, 결측치가 있는지 확인 → info()
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 2 to 44
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  119 non-null    float64
 1   sepal_width   120 non-null    float64
 2   petal_length  120 non-null    float64
 3   petal_width   120 non-null    float64
dtypes: float64(4)
memory usage: 4.7 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 93 to 55
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  29 non-null     float64
 1   sepal_width   30 non-null     float64
 2   petal_length  30 non-null     float64
 3   petal_width   30 non-null     float64
dtypes: float64(4)
memory usage: 1.2 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   species  120 non-

In [60]:
# x_train과 x_test의 기초통계량 비교
print(x_train.describe().T)
print(x_test.describe().T)
print(y_train.describe())

              count      mean        std  min    25%   50%    75%    max
sepal_length  119.0  5.920168   0.841667  4.3  5.150  6.00  6.500    7.9
sepal_width   120.0  4.295000  13.419100  2.2  2.800  3.00  3.400  150.0
petal_length  120.0  3.816667   1.798848  1.1  1.575  4.40  5.225    6.9
petal_width   120.0  1.226667   0.780512  0.1  0.300  1.35  1.800    2.5
              count      mean       std  min    25%   50%    75%  max
sepal_length   29.0  5.596552  0.709367  4.6  5.000  5.50  5.900  7.6
sepal_width    30.0  3.000000  0.522593  2.0  2.625  3.00  3.300  4.2
petal_length   30.0  3.523333  1.631518  1.0  1.600  4.05  4.925  6.6
petal_width    30.0  1.090000  0.685490  0.1  0.350  1.15  1.575  2.3
          species
count  120.000000
mean     0.666667
std      0.473381
min      0.000000
25%      0.000000
50%      1.000000
75%      1.000000
max      1.000000


In [61]:
# y데이터도 확인 (분류모델의 경우엔 value_counts() / 회귀 모델이면 describe())
print(y_train.head())
print(y_train.value_counts())

   species
0        0
1        0
2        1
3        1
4        1
species
1          80
0          40
Name: count, dtype: int64


### ✅ 3. 데이터 전처리 및 분리
#### 1) 결측치, 2) 이상치, 3) 변수 처리하기

In [62]:
# 결측치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
species    0
dtype: int64


In [63]:
# 결측치 제거
# df = df.dropna()
# print(df)

In [64]:
# 결측치 대체
# 연속형 변수 - 평균, 중앙값
# 평균 : df['변수명'].mean()
# 중앙값 : df['변수명'].median()

# 이산형 변수 : 최빈값
# df['변수명'].mode()

# 대체하기
# df['변수명'] = df['변수명'].fillna(대체할 값)

### ※결측치 대체시 주의사항 : train data의 중앙값(혹은 다른 대체할 값)으로 test data도 똑같이 na를 대체해야 함!!
모델의 일관성과 일반화를 유지하기 위해서.           
- 일관성 유지           
    - x_train 데이터셋은 모델이 학습하는 데이터인데, 이 데이터의 통계적 특성(중앙값, 평균 등)을 모델이 학습하는 동안 사용함. 따라서 x_train 데이터셋의 중앙값을 사용하면 모델이 훈련중에 본 데이터의 특성과 일치하게 됨.            
    - x_test는 모델을 평가할때 사용하는데 이의 결측치를 x_train의 통계값으로 대체하면 모델이 학습한 특성과 일치하는 방식으로 결측치를 처리하므로 훈련과 테스트 환경을 일관되게 유지함.
- 일반화 보장           
    - 데이터 누수 방지 : x_test의 중앙값을 사용하게 될 경우 테스트 데이터셋의 정보를 훈련 과정에 사용하게 되어 데이터 누수가 발생할 수 있음. 이는 모델이 실제로 새로운 데이터를 처리할 때 일반화 성능이 떨어질 가능성이 있으므로 x_train의 통계값을 사용하여 미연에 방지함.

→ 본 데이터의 특성을 유지하고 데이터의 누수를 방지하며 일반화 성능을 보장하기 위해 train data의 통계값을 test data의 결측치 대체 등에 사용한다.

Test data는 unseen data이기 때문에 훈련에 사용하지 않는다!! test data는 나중에 성능 평가 할때"만" 사용

In [65]:
# 결측치 대체 - 중앙값
median = x_train['sepal_length'].median()

x_train['sepal_length'] = x_train['sepal_length'].fillna(median)
x_test['sepal_length'] = x_test['sepal_length'].fillna(median)

In [66]:
# 이상치 확인
cond1 = (x_train['sepal_width'] >= 10)
print(len(x_train[cond1]))

1


In [67]:
# 이상치 대체 : np.where(조건, 조건에 해당시 대체할 값, 조건에 해당하지 않으면 사용할 값)
# (참고) df['변수명'] = np.where(df['변수명'] >=5, 대체할 값, df['변수명'])

# 예를 들어 'sepal_width' 값이 10이 넘으면 이상치라고 가정할 때 이상치를 제외한 Max 값을 구해서 대체하기.
# 내 코드 : x_train['sepal_width'] = np.where(x_train['sepal_width'] >= 10, x_train['sepal_width'].max(), x_train['sepal_width'])
# max() 함수의 경우 데이터 내의 모든 값(이상치나 결측치 포함) 중에서 최댓값을 계산하기 때문에 이상치를 제외하는 조건식을 써야 함.

# 이상치를 제외하는 조건식
cond1 = (x_train['sepal_width'] <= 10)
max_sw = x_train[cond1]['sepal_width'].max()            # 4.4
print(max_sw)
# 이상치를 제외한 max 값으로 이상치 대체하기
x_train['sepal_width'] = np.where(x_train['sepal_width'] > 10, max_sw, x_train['sepal_width'])
print(x_train.describe())

4.4
       sepal_length  sepal_width  petal_length  petal_width
count    120.000000   120.000000    120.000000   120.000000
mean       5.920833     3.081667      3.816667     1.226667
std        0.838155     0.429966      1.798848     0.780512
min        4.300000     2.200000      1.100000     0.100000
25%        5.175000     2.800000      1.575000     0.300000
50%        6.000000     3.000000      4.400000     1.350000
75%        6.500000     3.400000      5.225000     1.800000
max        7.900000     4.400000      6.900000     2.500000


In [68]:
# 변수처리
# 불필요한 변수 제거
# df = df.drop(columns = ['변수1','변수2']) # df = df.drop(['변수1','변수2'], axis=1)
# 필요시 변수 추가(파생변수 생성)
# df['파생변수명'] = df['A'] * df['B'] (파생변수 생성 수식)
# 원핫인코딩(가변수 처리)
# x_train = pd.get_dummies(x_train) 
# # x_test = pd.get_dummies(x_test)
# print(x_train.info())
# print(x_test.info())

#### 데이터 분리

In [69]:
# 데이터를 훈련 세트와 검증 세트로 분할 (훈련 80%, 검증 20%)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['species'], test_size=0.2, random_state=2023, stratify=y_train['species'])
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(96, 4)
(24, 4)
(96,)
(24,)


### ✅ 4. 모델링 및 성능평가

In [70]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [71]:
# 검증용 데이터로 예측
y_pred = model.predict(x_val)

In [72]:
# 성능 검증 : 정확도(accuracy_score), F1score(f1_score), AUC(roc_auc_score) // + recall_score, precision_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
acc = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
# 다중분류일 경우 f1_score(y_true, y_pred, average=’macro’)
auc = roc_auc_score(y_val, y_pred)

In [73]:
# 정확도 (Accuracy score)
print(acc)

1.0


In [74]:
# f1 score
print(f1)

1.0


In [75]:
# auc
print(auc)

1.0


In [76]:
# 참고 : 혼동행렬
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)
print(cm)

# ##### 예측
# ##### 0  1
# 실제 0 TN FP 
# 실제 1 FN TP

[[ 8  0]
 [ 0 16]]


### ✅ 5. 예측값 제출
#### (주의) x_test를 모델에 넣어 나온 예측값 제출

In [77]:
# (실기시험 안내사항)
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({ 'result': y_result }).to_csv('수험번호.csv', index=False)
# 모델을 사용하여 테스트 데이터 예측

# 1. 특정 클래스로 분류할 경우 : predict
y_result = model.predict(x_test)
print(y_result[:5])

# 2. 특정 클래스로 분류될 확률을 구할 경우 : predict_proba
y_result_prob = model.predict_proba(x_test)
print(y_result_prob[:5])

# 예측 결과 이해해보기
result_prob = pd.DataFrame({
    'result' : y_result,
    'prob_0' : y_result_prob[:, 0],
    'prob_1' : y_result_prob[:, 1]
})

# setosa 일 확률 : y_result_prob[:, 0]
# 그 외의 종일 확률 : y_result_prob[:, 1]
print(result_prob[:5])

[1 1 1 0 1]
[[0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.03 0.97]]
   result  prob_0  prob_1
0       1    0.00    1.00
1       1    0.00    1.00
2       1    0.00    1.00
3       0    1.00    0.00
4       1    0.03    0.97


In [ ]:
# ★tip : 데이터를 저장한다음 불러와서 제대로 제출했는지 확인해보자
# pd.DataFrame({'result': y_result}).to_csv('수험번호.csv', index=False) # df2 = pd.read_csv("수험번호.csv")
# print(df2.head())